<h1>Reconocimiento de audio</h1>

<b>Disclaimer: Este notebook utiliza funciones que estan definidos en distintos archivos .py que estan incluidos en la carpeta del proyecto <b>

<h2>Importacion de librerias necesarias</h2>

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from tensorflow.keras.regularizers import l2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import parselmouth
from audioLoading import loadAudios
from scipy.io import wavfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from convertFiles import convertFiles
from pathlib import Path
from scipy.io import wavfile
import pyaudio
import wave
import joblib

C:\Users\ADMIN\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<h2>Modelo 1</h2>

Cargar los audios

In [249]:
audios = loadAudios(f"./AUDIO/All-Audios/Metodo-01_Denoised/*")
audios_features = []

Extraccion de caracteristicas

In [251]:
audios_features = []
labels =[]
for audio_signal,filename in audios:
    mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
    mfcc_values = np.mean(mfccs, axis=1)
    features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
    labels.append(str.split(filename,'-')[0])
    audios_features.append(features)

Crear el DataFrame

In [253]:
df = pd.DataFrame(audios_features)
df["Label"]=labels
df = df[["Label"]+list(df.columns[:-1])]
df.to_csv("audio_features.csv",index=False)

Establecer X y Y

In [255]:
x = df.drop(columns=["Label"])
y = df["Label"]

Normalizar datos (Usando Standard Scaler)

In [257]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [259]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2, random_state=42)
print(x_train)

[[ 0.8325357   0.8192918  -0.6310274  ... -1.3693336  -0.8764015
  -0.17544797]
 [-0.621694    2.0198948  -1.4107885  ... -0.6495892  -1.1510254
  -0.34164265]
 [ 1.1462659   1.2306153  -0.22834    ... -0.47825438 -0.70871544
   0.20235932]
 ...
 [ 0.01998025 -1.0134624  -0.18002631 ...  1.2353414   1.201707
   0.9417349 ]
 [ 0.9267373  -0.29916263 -0.13627055 ... -0.43823695 -1.2847793
  -0.6475665 ]
 [-0.32974184  1.3893281   0.9636446  ... -0.74362266 -0.39500868
  -0.12389506]]


Crear y entrenar el modelo

In [261]:
clf = SVC(kernel='linear', C=10, gamma='scale',probability=True)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)

In [223]:
accuracy = clf.score(x_test,y_test)
print(accuracy)

0.949438202247191


In [263]:
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(accuracy)
joblib.dump(clf, "speaker_recognition_model_01.pkl")

[[ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  4  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0 

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['speaker_recognition_model_01.pkl']

Audio Recording

In [97]:
filename = "test_01.wav"
duration = 3
sr =22050
print("Recording...")

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=sr, input=True, frames_per_buffer=1024)
frames = []
for _ in range(0, int(sr / 1024 * duration)):
    data = stream.read(1024)
    frames.append(data)
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(filename, "wb")
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wf.setframerate(sr)
wf.writeframes(b"".join(frames))
wf.close()

print("Recording finished.")

Recording...
Recording finished.


In [269]:
test_features = []
labels =[]
audio_signal = librosa.load('./test_01.wav')

mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
mfcc_values = np.mean(mfccs, axis=1)
features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
test_features.append(features)

feature_df = pd.DataFrame(test_features,columns=x.columns)
if features is None:        
    print("No speech detected.")

model = joblib.load("speaker_recognition_model_01.pkl")
probs = model.predict_proba(feature_df.to_numpy())
speaker_labels = model.classes_
max_prob = max(probs[0])
best_speaker = speaker_labels[np.argmax(probs[0])]

if max_prob < 0.2:  
    best_speaker = "unknown"

print(f"Identified Speaker: {best_speaker} (Confidence: {max_prob:.2f})")

Identified Speaker: unknown (Confidence: 0.15)


<h2>Modelo 2</h2>

Cargar los audios

In [271]:
audios = loadAudios(f"./AUDIO/All-Audios/Metodo-02_Denoised/*")
audios_features = []

Extraccion de caracteristicas

In [273]:
audios_features = []
labels =[]
for audio_signal,filename in audios:
    mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
    mfcc_values = np.mean(mfccs, axis=1)
    features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
    labels.append(str.split(filename,'-')[0])
    audios_features.append(features)

Crear el DataFrame

In [275]:
df = pd.DataFrame(audios_features)
df["Label"]=labels
df = df[["Label"]+list(df.columns[:-1])]
df.to_csv("audio_features.csv",index=False)

Establecer X y Y

In [277]:
x = df.drop(columns=["Label"])
y = df["Label"]

Normalizar datos (Usando Standard Scaler)

In [279]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [281]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2, random_state=42)


Crear y entrenar el modelo

In [283]:
model2 = RandomForestClassifier(n_estimators=100, max_depth=32, random_state=42)
model2.fit(x_train,y_train)

RandomForestClassifier(max_depth=32, random_state=42)

In [285]:
accuracy = model2.score(x_test,y_test)
print(accuracy)

0.9705882352941176


In [287]:
y_pred = model2.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred,zero_division=True))
print(accuracy)
joblib.dump(model2, "speaker_recognition_model_02.pkl")

[[ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  5  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  

['speaker_recognition_model_02.pkl']

Audio Recording

In [245]:
filename = "test_01.wav"
duration = 3
sr =22050
print("Recording...")

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=sr, input=True, frames_per_buffer=1024)
frames = []
for _ in range(0, int(sr / 1024 * duration)):
    data = stream.read(1024)
    frames.append(data)
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(filename, "wb")
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wf.setframerate(sr)
wf.writeframes(b"".join(frames))
wf.close()

print("Recording finished.")

Recording...
Recording finished.


In [289]:
test_features = []
labels =[]
audio_signal = librosa.load('./test_01.wav')

mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
mfcc_values = np.mean(mfccs, axis=1)
features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
test_features.append(features)

feature_df = pd.DataFrame(test_features,columns=x.columns)
if features is None:        
    print("No speech detected.")

model = joblib.load("speaker_recognition_model_02.pkl")
probs = model.predict_proba(feature_df.to_numpy())
speaker_labels = model.classes_
max_prob = max(probs[0])
best_speaker = speaker_labels[np.argmax(probs[0])]

if max_prob < 0.2:  
    best_speaker = "unknown"

print(f"Identified Speaker: {best_speaker} (Confidence: {max_prob:.2f})")

Identified Speaker: unknown (Confidence: 0.11)


<h2>Modelo 3</h2>

Cargar los audios

In [291]:
audios = loadAudios(f"./AUDIO/All-Audios/Metodo-03_Denoised/*")
audios_features = []

Extraccion de caracteristicas

In [293]:
audios_features = []
labels =[]
for audio_signal,filename in audios:
    mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
    mfcc_values = np.mean(mfccs, axis=1)
    features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
    labels.append(str.split(filename,'-')[0])
    audios_features.append(features)

Crear el DataFrame

In [298]:
df = pd.DataFrame(audios_features)
df["Label"]=labels
df = df[["Label"]+list(df.columns[:-1])]
df.to_csv("audio_features.csv",index=False)

Establecer X y Y

In [300]:
x = df.drop(columns=["Label"])
y = df["Label"]

Normalizar datos (Usando Standard Scaler)

In [302]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [304]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2, random_state=42)
print(x_train)

[[ 1.1610744   0.9109248  -1.1013682  ...  0.05935714 -0.00837376
  -1.0416665 ]
 [ 1.2457147   0.37551764 -0.73893654 ...  1.3499959   0.6623107
   0.815851  ]
 [-1.2578881   0.47306472  1.1283686  ... -1.1779135  -0.92506033
  -0.8852448 ]
 ...
 [-1.4525996  -0.74860686  1.5889126  ... -0.6789294  -0.28512546
  -0.27053785]
 [-0.2902485  -0.48539948  0.07966585 ... -1.2966229  -0.41573796
   0.4302839 ]
 [-0.49466035 -1.1452633   0.35815242 ...  0.8378222   1.0334148
   0.16535828]]


Crear y entrenar el modelo

In [306]:
model3 = RandomForestClassifier(n_estimators=200, max_depth=32, random_state=42)
model3.fit(x_train,y_train)

RandomForestClassifier(max_depth=32, n_estimators=200, random_state=42)

In [308]:
accuracy = model3.score(x_test,y_test)
print(accuracy)

0.9465648854961832


In [23]:
y_pred = model3.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred,zero_division=True))
print(accuracy)
joblib.dump(model3, "speaker_recognition_model_03.pkl")

[[5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 

['speaker_recognition_model_03.pkl']

Audio Recording

In [25]:
filename = "test_01.wav"
duration = 3
sr =22050
print("Recording...")

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=sr, input=True, frames_per_buffer=1024)
frames = []
for _ in range(0, int(sr / 1024 * duration)):
    data = stream.read(1024)
    frames.append(data)
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(filename, "wb")
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wf.setframerate(sr)
wf.writeframes(b"".join(frames))
wf.close()

print("Recording finished.")

Recording...
Recording finished.


In [312]:
test_features = []
labels =[]
audio_signal = librosa.load('./test_01.wav')

mfccs = librosa.feature.mfcc(y=audio_signal[0], sr=audio_signal[1], n_mfcc=30)
mfcc_values = np.mean(mfccs, axis=1)
features = {f"mfcc_{i+1}":mfcc_values[i] for i in range(30)}
test_features.append(features)

feature_df = pd.DataFrame(test_features,columns=x.columns)

if features is None:        
    print("No speech detected.")

loadedModel = joblib.load("speaker_recognition_model_03.pkl")
probs = loadedModel.predict_proba(feature_df.to_numpy())
speaker_labels = loadedModel.classes_

max_prob = max(probs[0])
best_speaker = speaker_labels[np.argmax(probs[0])]

if max_prob < 0.2:  
    best_speaker = "unknown"
print(f"Identified Speaker: {best_speaker} (Confidence: {max_prob:.2f})")

Identified Speaker: unknown (Confidence: 0.15)
